# Statistics and Analysis of Characters and Anime 

In [50]:
#Import libraries
import pandas as pd
import numpy as np
import datetime
import ast

from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler

import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap


%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
plt.style.use('ggplot')

In [51]:
# Characters who appear >10 times in the fanfiction, their names gathered into a in a csv file. Converted to pandas DF.
characters = pd.read_csv('myhero_names_rev.csv')

# Archive of Our Own
AO3 = pd.read_csv('AO3.csv')

# Series overview: Original season number, Number of episodes, First aired, and Last aired.
og_season_anime_airdate = pd.read_csv('og_air_clean.csv')

# Season 1 Episode number overall, Episode number in the season, Title, Storyboarded by, Directed by,
#     Written by, Original air date, and English air date.
season_one_anime = pd.read_csv('season_one_clean.csv')

# Season 2-6 Episode number overall, Episode number in the season, Title, Storyboarded by, Directed by,
#     Written by, Viewership rating, Original air date, and English air date.
season_two_anime = pd.read_csv('season_two_clean.csv')
season_three_anime = pd.read_csv('season_three_clean.csv')
season_four_anime = pd.read_csv('season_four_clean.csv')
season_five_anime = pd.read_csv('season_five_clean.csv')
season_six_anime = pd.read_csv('season_six_clean.csv')

# Original Video Animation (OVA) titled "Save! Rescue Training!"
# Number (8 total released), Title, Storyboard by, Directed by, Written by, Original release date.
og_vid_animation_anime =  pd.read_csv('OVAs_clean.csv')

# Japan Home Video Release
# Season, Volume, Epsiodes, Release date.
Japan_homevid_release_anime = pd.read_csv('Japan_HV_clean.csv')

# Funimation/Crunchyroll (North America)
# Season, Volume, Episodes, Release date.
funimation_crunchyroll_anime = pd.read_csv('Fun_Crunch_NA_clean.csv')

# Universal Pictures/Manga Entertainment (UK and Ireland)
# Season, Volume, Episodes, Release date.
universalpictures_manga_anime = pd.read_csv('Up_Uk_clean.csv')

# Universal Sony/Madman Entertainment (Australia and New Zealand)
# Season, Volume, Episodes, Release date Universal, Release date Madman.
universalsony_madman_anime = pd.read_csv('Us_Me_clean.csv')

In [52]:
name_1 = [x for x in characters['1'].values]
name_2 = [x for x in (characters['2'].dropna().values)]
mhanames = name_1 + name_2


In [53]:
dont_want = ['Parents', 'Suffering', 'Loss Of His Will', 'Niece', 'mentioned', 'Family', 'Notebooks', 'Character', 'quirk', 'Friend', 'Eriks', 'Werewolf', 'Prototype', 'Mother', 'Sibling', 'Sidekicks', 'l Jade', 'Nezuko']

In [54]:
AO3['ships'] = AO3['ships'].apply(ast.literal_eval)

In [55]:
Ships = AO3.explode('ships')['ships'].value_counts()

In [56]:
characters['First'] = characters['1']
characters = characters.set_index('First')

In [61]:
def pullship(char1, char2):
    name = []
    name2 = []
    try:
        for i in characters.loc[char1]:
            if type(i) != float:
                name.append(i)
    except:
        name.append(char1)
        
    try:
        for i in characters.loc[char2]:
            if type(i) != float:
                name2.append(i)
    except:
        name2.append(char2)
    ship = [name, name2]
    shipname = char1 + ' & ' + char2
    AO3[shipname] = 0
    for index, row in AO3.iterrows():
        for name in ship[0]:
            if row[shipname] == 0:
                if name in ''.join(row['ships']):
                    for name2 in ship[1]:
                        if name2 in ''.join(row['ships']):
                            AO3.loc[index, shipname] = 1


In [62]:
display(Ships[25:].head())
len(Ships)

Midoriya Izuku/Shinsou Hitoshi           40
Bakugou Katsuki & Kirishima Eijirou      39
Midoriya Izuku/Reader                    34
Sero Hanta/Todoroki Shouto               34
Midoriya Izuku & Sensei | All For One    34
Name: ships, dtype: int64

2140

In [63]:
pullship('Deku', 'Katsuki')
pullship('Aizawa', 'Yamada Hizashi')
pullship('Aizawa', 'Deku')
pullship('Katsuki', 'Enjiro')
pullship('Deku', 'Todoroki Sho')
pullship('Dabi', 'Keigo')
pullship('Deku', 'Uraraka')
pullship('Deku', 'All Might')
pullship('Deku', 'Class')
pullship('Denki', 'Shinsou Hitoshi')
pullship('Deku', 'Shinsou Hitoshi')
pullship('Katsuki', 'Reader')
pullship('Katsuki', 'Todoroki Sho')
pullship('Deku', 'Yamada Hizashi')
pullship('Mina', 'Eijirou')
pullship('Jirou Kyouka', 'Denki')
pullship('Deku', 'Shigaraki Tomura')
pullship('Aizawa', 'Shinsou Hitoshi')
pullship('Deku', 'Nedzu')
pullship('Deku', 'Reader')

In [66]:
AO3 = AO3.rename(columns = {'Aizawa & Shinsou Hitoshi': 'Aizawa & Shinsou',
                            'Jirou Kyouka & Denki': 'Jirou & Denki', 'Mina & Eijirou': 'Mina & Kirashima', 'Deku & Yamada Hizashi': 'Deku & Prez Mic', 'Katsuki & Todoroki Sho': 'Bakugo & Shoto', 'Katsuki & Reader': 'Bakugo and Reader', 'Deku & Shinsou Hitoshi': 'Deku & Shinsou', 'Denki & Shinsou Hitoshi': 'Denki & Shinsou', 'Dabi & Keigo': 'Dabi & Hawks', 'Deku & Todoroki Sho': 'Deku & Shoto', 'Katsuki & Enjiro': 'Bakugo & Kirishima', 'Deku & Katsuki': 'Deku & Bakugo', 'Aizawa & Yamada Hizashi': 'Aizawa & Prez Mic'})


In [67]:
AO3

,title,mixed fandom,ships,characters,hits,kudos,published,Deku & Bakugo,Aizawa & Prez Mic,Aizawa & Deku,...,Deku & Shinsou,Bakugo and Reader,Bakugo & Shoto,Deku & Prez Mic,Mina & Kirashima,Jirou & Denki,Deku & Shigaraki Tomura,Aizawa & Shinsou,Deku & Nedzu,Deku & Reader
0,Purple poison flames meet Blue fire,['Fandoms:\n鬼滅の刃 | Demon Slayer: Kimetsu no Ya...,[Dabi | Todoroki Touya & Kochou Shinobu],"['Dabi | Todoroki Touya', 'Kochou Shinobu']",0,NaN,02 Apr 2023,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Stuck With Babysitter Duty,['Fandoms:\n僕のヒーローアカデミア | Boku no Hero Academi...,[Dazai Osamu/Nakahara Chuuya (Bungou Stray Dogs)],"['Midoriya Izuku', 'Dazai Osamu (Bungou Stray ...",0,NaN,02 Apr 2023,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Power in UA!!!,"['Fandoms:\nChainsaw Man (Anime), 僕のヒーローアカデミア ...","[Bakugou Katsuki/Power, Power & Class 1-A]","['Power (Chainsaw Man)', 'Ashido Mina', 'Iida ...",508,7.0,02 Apr 2023,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Balance and Bondeds,['Fandoms:\n僕のヒーローアカデミア | Boku no Hero Academi...,[Aizawa Shouta | Eraserhead/Yamada Hizashi | P...,"['Midoriya Izuku', 'Midoriya Inko', 'Bakugou K...",11924,481.0,02 Apr 2023,0,1,1,...,1,0,0,1,0,0,0,1,0,0
4,Can't Hold Our Babies,['Fandoms:\n僕のヒーローアカデミア | Boku no Hero Academi...,[Bakugou Katsuki/Midoriya Izuku/Uraraka Ochako...,"['Bakugou Katsuki', 'Uraraka Ochako', 'Midoriy...",0,NaN,02 Apr 2023,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Seeing double,['Fandoms:\n僕のヒーローアカデミア | Boku no Hero Academi...,"[Midoriya Izuku/Usagiyama Rumi | Miruko, Midor...","['Midoriya Izuku', 'Class 1-A (My Hero Academi...",14960,507.0,28 Mar 2023,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1996,My Idol,['Fandoms:\n僕のヒーローアカデミア | Boku no Hero Academi...,[Bakugou Katsuki/Midoriya Izuku],"['Bakugou Katsuki', 'Midoriya Izuku']",3348,234.0,28 Mar 2023,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1997,Woodland Paths,['Fandoms:\n僕のヒーローアカデミア | Boku no Hero Academi...,"[Midoriya Izuku & Yagi Toshinori | All Might, ...","['Midoriya Izuku', 'Yagi Toshinori | All Might...",4604,244.0,28 Mar 2023,1,0,0,...,0,0,1,0,0,0,0,0,0,0
1998,Boku No Hero Academia: Ones Way,['Fandoms:\n僕のヒーローアカデミア | Boku no Hero Academi...,[],"['Mia Ikati', 'Rue Achlys', 'Ryuuji Takao', 'I...",44,NaN,28 Mar 2023,0,0,0,...,0,0,0,0,0,0,0,0,0,0
